# SQL Agent v3 — Interactive Notebook

This notebook is a **thin wrapper** around `sql_agent_v3.py`.  
All agent logic lives in that file so it can also be imported by `benchmark_chinook.py`.

```
D:/SQL_agent/
├── sql_agent_v3.py        ← agent logic (the real code)
├── sql_agent_v3.ipynb     ← this file (interactive wrapper)
└── benchmark_chinook.py   ← imports sql_agent_v3
```

**Override any setting via env vars before launching Jupyter:**
```powershell
$env:SQL_AGENT_DB_PATH    = 'D:/SQL_agent/Chinook_Sqlite.sqlite'
$env:SQL_AGENT_CHROMA_DIR = 'D:/SQL_agent/chroma_sql_rag'
$env:SQL_AGENT_MODEL      = 'qwen2.5-coder:7b'
$env:OLLAMA_HOST          = 'http://localhost:11434'
jupyter notebook
```

In [1]:
# All logic lives in sql_agent_v3.py — import everything from there.
# Make sure this notebook is in the same directory as sql_agent_v3.py,
# or add the directory to sys.path first.
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

from sql_agent_v3 import (
    DB_PATH, CHROMA_DIR, MODEL_NAME,
    run_sql_agent, app,
    build_or_load_chroma, get_database_schema,
)

print(f"DB_PATH    : {DB_PATH}")
print(f"CHROMA_DIR : {CHROMA_DIR}")
print(f"MODEL      : {MODEL_NAME}")

d:\Anaconda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DB_PATH    : D:/SQL_agent/Chinook_Sqlite.sqlite
CHROMA_DIR : d:\SQL_agent\chroma_sql_rag
MODEL      : qwen2.5-coder:7b


In [2]:
# ── Run a question ────────────────────────────────────────────────────────────
question = "Show me the top 5 customers who spent the most money, including their email."

out = run_sql_agent(question)

print("-" * 60)
print("Generated SQL:")
print(out["sql"])
print("-" * 60)

if out["error"]:
    print(f"❌ Error: {out['error']}")
else:
    result = out["result"] or {"columns": [], "rows": []}
    print(result["columns"])
    for row in result["rows"]:
        print(row)

✅ Chroma collection already has 13 docs.
------------------------------------------------------------
Generated SQL:
SELECT c.FirstName, c.LastName, c.Email, SUM(i.Total) AS TotalSpent FROM Customer c JOIN Invoice i ON c.CustomerId = i.CustomerId GROUP BY c.CustomerId ORDER BY TotalSpent DESC LIMIT 5;
------------------------------------------------------------
['FirstName', 'LastName', 'Email', 'TotalSpent']
('Helena', 'Holý', 'hholy@gmail.com', 49.62)
('Richard', 'Cunningham', 'ricunningham@hotmail.com', 47.62)
('Luis', 'Rojas', 'luisrojas@yahoo.cl', 46.62)
('Ladislav', 'Kovács', 'ladislav_kovacs@apple.hu', 45.62)
('Hugh', "O'Reilly", 'hughoreilly@apple.ie', 45.62)


In [3]:
# ── Inspect full state ────────────────────────────────────────────────────────
print("RAG context retrieved:")
print(out["rag_context"][:800], "..." if len(out.get("rag_context", "")) > 800 else "")

RAG context retrieved:
CREATE TABLE users (
            id INTEGER PRIMARY KEY,
            username TEXT NOT NULL,
            email TEXT,
            signup_date DATE
        );

CREATE TABLE [Invoice]
(
    [InvoiceId] INTEGER  NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Invoice] PRIMARY KEY  ([InvoiceId]),
    FOREIGN KEY ([CustomerId]) REFERENCES [Customer] ([CustomerId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(2 ...


In [4]:
# ── View full schema ──────────────────────────────────────────────────────────
print(get_database_schema(DB_PATH))

CREATE TABLE [Album]
(
    [AlbumId] INTEGER  NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),
    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);
CREATE TABLE [Artist]
(
    [ArtistId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])
);
CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),
    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([EmployeeId]) 
		ON DELETE NO ACTION O